In [1]:
import folium
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon
import pandas as pd

In [2]:
# Read the dataset after pre-processing
df=pd.read_csv('Falcon9.csv')

In [3]:
# All launch site are located ay USA, so I choose the location central of America to overview the map.
usa_central = [34.159181, -85.674742]
map1 = folium.Map(location=usa_central, zoom_start=4)
map1

In [4]:
# Search for the location of all launch site on the map, and mark their name on their location.
# Create a new dataset contains launch sites along with their coordinates.
df1 = df[['LaunchSite', 'Latitude', 'Longitude']]
for index, coordinates in df1.iterrows():
    circle = folium.Circle([coordinates['Latitude'], coordinates['Longitude']], color='red', radius=500).add_to(map1)
    marker = folium.Marker([coordinates['Latitude'], coordinates['Longitude']], icon=DivIcon(html=coordinates['LaunchSite'])).add_to(map1)
map1

In [5]:
# Create a new attribute. If landing label is 1, color will be green. If landing label is 0, color will be red.
landing=[1]
color_label=[]
for outcome in df['Landing']:
    if outcome in landing:
        color_label.append('green')
    else:
        color_label.append('red')
df['LabelColor']=color_label
df

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Landing,LabelColor
0,6,2010-06-04,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1,0,0,0,NaN,1.0,0,B0003,-80.577366,28.561857,0,red
1,8,2012-05-22,Falcon 9,525.0,LEO,CCSFS SLC 40,None None,1,0,0,0,NaN,1.0,0,B0005,-80.577366,28.561857,0,red
2,10,2013-03-01,Falcon 9,677.0,ISS,CCSFS SLC 40,None None,1,0,0,0,NaN,1.0,0,B0007,-80.577366,28.561857,0,red
3,11,2013-09-29,Falcon 9,500.0,PO,VAFB SLC 4E,False Ocean,1,0,0,0,NaN,1.0,0,B1003,-120.610829,34.632093,0,red
4,12,2013-12-03,Falcon 9,3170.0,GTO,CCSFS SLC 40,None None,1,0,0,0,NaN,1.0,0,B1004,-80.577366,28.561857,0,red
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,102,2020-09-03,Falcon 9,15600.0,VLEO,KSC LC 39A,True ASDS,2,1,1,1,5e9e3032383ecb6bb234e7ca,5.0,12,B1060,-80.603956,28.608058,1,green
86,103,2020-10-06,Falcon 9,15600.0,VLEO,KSC LC 39A,True ASDS,3,1,1,1,5e9e3032383ecb6bb234e7ca,5.0,12,B1058,-80.603956,28.608058,1,green
87,104,2020-10-18,Falcon 9,15600.0,VLEO,KSC LC 39A,True ASDS,6,1,1,1,5e9e3032383ecb6bb234e7ca,5.0,12,B1051,-80.603956,28.608058,1,green
88,105,2020-10-24,Falcon 9,15600.0,VLEO,CCSFS SLC 40,True ASDS,3,1,1,1,5e9e3033383ecbb9e534e7cc,5.0,12,B1060,-80.577366,28.561857,1,green


In [6]:
# Mark success or failed launches for each site with marker cluster using color schemes.
marker_cluster = MarkerCluster()
map1.add_child(marker_cluster)

for index, coordinates in df.iterrows():
    marker = folium.Marker([coordinates['Latitude'], coordinates['Longitude']], 
                  icon=folium.Icon(icon_color=coordinates['LabelColor']))
    marker_cluster.add_child(marker)
map1


In [7]:
# Get the coordinates of any point on the map by the mouse position using folium - MousePosition
formatter = "function(num) {return L.Util.formatNum(num, 6) + ' º ';};"

MousePosition(
    position="topright",
    separator=" , ",
    empty_string="NaN",
    lng_first=False,
    num_digits=20,
    prefix="Coordinates:",
    lat_formatter=formatter,
    lng_formatter=formatter,
).add_to(map1)

map1

In [8]:
from math import radians, sqrt, asin, sin, cos

In [9]:
R = 6371 # Earth radius
def geographic_distance(lat1, log1, lat2, log2):
    # Convert decimal to radian
    lat1, log1, lat2, log2 = map(radians, [lat1, log1, lat2, log2])
    dlog = log2 - log1
    dlat = lat2 - lat1
    # Haversine formula
    h = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlog / 2)**2
    distance = 2 * asin(sqrt(h)) * R
    return round(distance, 2)


In [10]:
# CCSFS SLC 40 launch site
launch_site_lat = 28.563197
launch_site_lon = -80.576820
# Florida East Coast Railway
closest_railway_lat = 28.572010
closest_railway_lon = -80.585318
# Samuel C Phillips Pkwy
closest_highway_lat = 28.563228
closest_highway_lon = -80.570759
# Melbourne
closest_city_lat = 28.097424
closest_city_lon = -80.642395
# Coastline
closest_coastline_lat = 28.563906
closest_coastline_lon = -80.568066

distance_railway = geographic_distance(launch_site_lat, launch_site_lon, closest_railway_lat, closest_railway_lon)
distance_highway = geographic_distance(launch_site_lat, launch_site_lon, closest_highway_lat, closest_highway_lon)
distance_city = geographic_distance(launch_site_lat, launch_site_lon, closest_city_lat, closest_city_lon)
distance_coastline = geographic_distance(launch_site_lat, launch_site_lon, closest_coastline_lat, closest_coastline_lon)

print('closest rail way: ',distance_railway,'KM')
print('closest high way: ',distance_highway,'KM')
print('closest city: ',distance_city,'KM')
print('closest coastline: ',distance_coastline,'KM')

closest rail way:  1.28 KM
closest high way:  0.59 KM
closest city:  52.19 KM
closest coastline:  0.86 KM


In [11]:
# Railway
distance_marker = folium.Marker([closest_railway_lat, closest_railway_lon], icon=DivIcon(html="{}KM".format(distance_railway))).add_to(map1)
coordinates = [[launch_site_lat,launch_site_lon],[closest_railway_lat,closest_railway_lon]]
lines=folium.PolyLine(locations=coordinates, weight=1).add_to(map1)
# Highway
distance_marker1 = folium.Marker([closest_highway_lat, closest_highway_lon], icon=DivIcon(html="{}KM".format(distance_highway))).add_to(map1)
coordinates1 = [[launch_site_lat,launch_site_lon],[closest_highway_lat,closest_highway_lon]]
lines1=folium.PolyLine(locations=coordinates1, weight=1).add_to(map1)
# City
distance_marker2 = folium.Marker([closest_city_lat, closest_city_lon], icon=DivIcon(html="{}KM".format(distance_city))).add_to(map1)
coordinates2 = [[launch_site_lat,launch_site_lon],[closest_city_lat,closest_city_lon]]
lines2 = folium.PolyLine(locations=coordinates2, weight=1).add_to(map1)
# Coastline
distance_marker3 = folium.Marker([closest_coastline_lat, closest_coastline_lon], icon=DivIcon(html="{}KM".format(distance_coastline))).add_to(map1)
coordinates3 = [[launch_site_lat,launch_site_lon],[closest_coastline_lat,closest_coastline_lon]]
lines3 = folium.PolyLine(locations=coordinates3, weight=1).add_to(map1)

map1